# Condensed rates vs. experiments
## Zhang & Winfree 2009, Dabby 2013, Genot et al. 2011

# Initialization - Global

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import log10, sqrt

sns.set(style="darkgrid")

# Helper function to calculate Euclidean distance
def euclid(x, y, df, take_log=False):
    if take_log:
        return sqrt(sum((log10(df[x])-log10(df[y]))**2))
    else:
        return sqrt(sum((df[x]-df[y])**2))
    
def sema(vect, fig):
    #semantics=', '.join('{}={}'.format(x,y) for (x,y) in fig.pepperargs['default'].items() if x != 'conc')
    nv = []
    for line in vect:
        v = []
        for (x,y) in fig.pepperargs[line].items():
            if x == 'conc': continue
            if x == 'max_complex_size': continue
            if x == 'max_complex_count': continue
            if x == 'max_reaction_count': continue
            if x == 'condensed': 
                z = x if y is True else 'detailed'
            else:
                z = '{}={}'.format(x,y)
            v.append(z)
        nv.append(', '.join(v))
    return nv
 

## Initialization - Figure 1

In [ ]:
from zhang2009_rates import data; z09r = data()
from dabby2013_rates import data; d13r = data()

Table1 = pd.DataFrame()

### Zhang and Winfree 2009

In [ ]:
for fig in z09r:
    fig.eval()
    #display(fig.pepperargs['default'])
    semantics=', '.join('{}={}'.format(x,y) for (x,y) in fig.pepperargs['default'].items() if x != 'conc')
    for df in fig.get_dataframes():
        df['Semantics']=semantics
        df.to_latex(fig.fname+'.tex', index=False, float_format='{:.3f}'.format)
        df['Name']=fig.name 
        #display(df)
        dist = euclid('Rate (experiment)', 'Rate (calculated)', df, take_log=True)
        print('{:s} -- Fitness: {:g}'.format(fig.fname, dist))
        Table1 = Table1.append(df)


### Dabby 2013

In [ ]:
for fig in d13r:
    fig.pepperargs['default']['conc'] = 'M'
    display(fig.pepperargs['default'])
    fig.eval()
    semantics=', '.join('{}={}'.format(x,y) for (x,y) in fig.pepperargs['default'].items() if x != 'conc')
    for df in fig.get_dataframes():
        df['Semantics']=semantics
        df.to_latex(fig.fname+'.tex', float_format='{:.3f}'.format, index=False)
        df['Name']=fig.name 
        display(df)
        dist = euclid('Rate (experiment)', 'Rate (calculated)', df, take_log=True)
        print('{:s} -- Fitness: {:g}'.format(fig.fname+'.tex', dist))
        Table1 = Table1.append(df)        

In [ ]:
# Data adjustments:
tmpfig = Table1.copy()
tmpfig['Rate (experiment)'] = log10(tmpfig['Rate (experiment)'])
tmpfig['Rate (calculated)'] = log10(tmpfig['Rate (calculated)'])
#display(tmpfig)

# Figure layout:
g = sns.lmplot(x="Rate (experiment)", y="Rate (calculated)", col="Name", hue='Name', data=tmpfig, sharex = False, sharey = False,
           col_wrap=2, ci=None, truncate=False, palette="muted", height = 3.6, scatter_kws={"s": 20, "alpha": 1})

(mi, ma)=(-2, 8)
plt.xlim(-2.5, 8.5)
plt.ylim(-2.5, 8.5)

for ax in g.axes:
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x,y: "$10^{{ {:.0f} }}$".format(x)))
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x,y: "$10^{{ {:.0f} }}$".format(x)))

    ax.plot([mi, ma], [mi, ma], linewidth=3, color='white', zorder=0)
    ax.set(xlabel='Experimental rate constant [/M/s]')
    ax.set(ylabel='Peppercorn rate constant [/M/s]')
    
g.set_titles(row_template="{row_name}", col_template="{col_name}")

plt.savefig('proximal_rates.pdf')
plt.savefig('proximal_rates.svg')

# Genot et al. 2011

In [ ]:
from genot2011 import data; g11 = data()
Table2 = pd.DataFrame()

In [ ]:
for e, fig in enumerate(g11):
    #display(fig.pepperargs['default'])
    fig.eval('default', cmpfig='hack', verbose=0)

    #fig.pepperargs['detailed-default'] = fig.pepperargs['default'].copy()
    #fig.pepperargs['detailed-default']['condensed'] = False
    #fig.eval('detailed-default', cmpfig='hack', verbose=0)

    #fig.pepperargs['ddG'] = fig.pepperargs['default'].copy()
    #fig.pepperargs['ddG']['dG_bp'] = -1.3
    #fig.eval('ddG', cmpfig='hack', verbose=0)

    for df in fig.get_dataframes():
        df['Semantics']=sema(df['Semantics'], fig)
        df['Name']=fig.name
        dist = euclid('Time (experiment)', 'Time (simulation)', df, take_log=True)
        print('{:s} -- Fitness: {:g}'.format(fig.name,dist))
        Table2 = Table2.append(df)

In [ ]:
#display(Table2)
Table2.to_latex('Genot2011data.tex', index=False, float_format='{:.3g}'.format)

tmpfig = Table2.copy()
tmpfig['Time (experiment)'] = log10(tmpfig['Time (experiment)'])
tmpfig['Time (simulation)'] = log10(tmpfig['Time (simulation)'])
#display(tmpfig)

sns.set_palette("husl")

mycolors = ['blue', 
            'red', 
            'green', 
            'orange', 
            'maroon', 
            'springgreen', 
            'cyan', 
            'magenta', 
            'yellow']
mycolors += list('kkkkkkkkkkk')
mymarker = list('po.*^v+_,phD8|_oooooooo')

g = sns.lmplot(x="Time (experiment)", y="Time (simulation)", hue='Name', height = 3.6, #col="Semantics", 
               data=tmpfig, fit_reg=False, markers=['o', 'x', 'x', 'v', 'v', 'o'], legend=False)
g.fig.set_size_inches(4.3,3.4)

# Adjust plot
(mi, ma)=(0, 5)
plt.ylim(-.5, 8.)

plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x,y: "$10^{{ {:.0f} }}$".format(x)))
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x,y: "$10^{{ {:.0f} }}$".format(x)))

for ax in plt.gcf().get_axes():
    ax.plot([mi, ma], [mi, ma], linewidth=3, color='white',zorder=0)

g.set(title='Genot et al. (2011)\nremote-toehold strand displacement')  
g.set(xlabel='Experimental completion time [s]')  
g.set(ylabel='Peppercorn completion time [s]')
    
legend = plt.legend(ncol=1, loc='upper left', fontsize=7.8);
frame = legend.get_frame()
frame.set_facecolor('white')

plt.savefig('remote_rates.pdf', bbox_inches='tight')
plt.savefig('remote_rates.svg', bbox_inches='tight')